# Import Library & Dataset

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder

from func import outlier_counter, get_all_univariate_outlier_index
from modelling_purpose import Xy, algorithm_report_accumulation

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('csv/imputed.csv')

# With / Without Outlier

Kita akan membuat 2 dataset dari `imputed.csv`, yaitu:
- Dengan Outlier
- Tanpa Outlier (Univariate)

In [3]:
df_without_outlier = df.copy()
outlier_columns  = ['TotalWorkingYears', 'YearsAtCompany', 'YearsInCurrentRole',
                    'YearsSinceLastPromotion', 'YearsWithCurrManager', 'TrainingTimesLastYear',
                    'NumCompaniesWorked', 'MonthlyIncome']
outlier_index = get_all_univariate_outlier_index(df_without_outlier, outlier_columns)
df_without_outlier.drop(df_without_outlier.index[outlier_index], inplace=True)

In [4]:
df.shape

(1029, 31)

In [5]:
df_without_outlier.shape

(650, 31)

`df` adalah dataset dengan outlier. Dan `df_without_outlier` adalah dataset tanpa outlier.

# Feature Engineering

In [6]:
X = df.drop('Attrition',axis=1)
y = df['Attrition'].map({'Yes':1,'No':0})
X_wo = df_without_outlier.drop('Attrition',axis=1)
y_wo = df_without_outlier['Attrition'].map({'Yes':1,'No':0})

categorical_features = X.select_dtypes(include='O').columns.tolist()
categorical_features

['BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'OverTime']

Semua categorical features, entah nominal ataupun ordinal akan digabung dan diaplikasikan dengan One-Hot Encoder.
Alasan mengapa ordinal features juga menggunakan One-Hot Encoder adalah karena pada kasus classifier, ordinal feature yang mempunyai continuous behaviour tidak berpengaruh seperti pada kasus regressor. Melakukan pe-ranking-an pada suatu feature menjadi tidak berguna.

In [7]:
X = pd.get_dummies(X, columns=categorical_features, drop_first=True)
X_wo = pd.get_dummies(X_wo, columns=categorical_features, drop_first=True)

# Model

Pada classifier problem, kita akan memilih salah satu metrik penilaian yang akan dijadikan acuan. Ini dikarenakan False Positive dan False Negative akan selalu trade-off satu sama lain. Jadinya, umumnya kita akan dihadapkan dengan 2 pilihan berikut:
- Kasus False Negative lebih beresiko daripada kasus False Positive
- Kasus False Positive lebih beresiko daripada kasus False Negative

Di kasus ini, False Positive dan False Negative bisa diterjemahkan seperti ini:
- FP : Pegawai yang tidak keluar, terprediksi keluar.
- FN : Pegawai yang keluar, terprediksi tidak keluar.

Untuk kasus Attrition, saya menganggap kasus **False Negative adalah yang beresiko**.

Alasannya adalah, apabila ada pegawai keluar namun terprediksi tidak keluar, perusahaan beresiko kehilangan pegawai potensialnya. Dengan decision seperti ini, maka saya putuskan untuk memilih **Recall** sebagai metric yang diutamakan.

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [9]:
algorithm_list = [LogisticRegression,DecisionTreeClassifier,RandomForestClassifier, GradientBoostingClassifier]

a = algorithm_report_accumulation(algorithm_list, X, y, .2, 'with Outliers')
b = algorithm_report_accumulation(algorithm_list, X_wo, y_wo, .2, 'without Outliers')
x = pd.concat([a, b],ignore_index=True)
x.set_index('Algorithm')

,Train Recall,Test Recall,Notes
Algorithm,,,
<class 'sklearn.linear_model._logistic.LogisticRegression'>,0.014184,0.000000,with Outliers
<class 'sklearn.tree._classes.DecisionTreeClassifier'>,1.000000,0.285714,with Outliers
<class 'sklearn.ensemble._forest.RandomForestClassifier'>,1.000000,0.085714,with Outliers
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>,0.858156,0.342857,with Outliers
<class 'sklearn.linear_model._logistic.LogisticRegression'>,0.144444,0.136364,without Outliers
<class 'sklearn.tree._classes.DecisionTreeClassifier'>,1.000000,0.500000,without Outliers
<class 'sklearn.ensemble._forest.RandomForestClassifier'>,1.000000,0.227273,without Outliers
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>,0.933333,0.409091,without Outliers


**Temuan** :
- LogisticRegression dengan default parameter underfit dengan parah. Entah di dataset dengan dan tanpa outlier, recall score tidak sampai 0.15.
- Model lainnya (selain LogisticRegression) overfit.

Dengan temuan ini, saya tidak melanjutkan untuk menggunakan LogisticRegression. Dengan asumsi bahwa dengan score serendah itu, tentunya akan memerlukan effort lebih untuk menaikkan scorenya, meskipun menggunakan Hyperparameter Tuning. Karena pada umumnya lebih mudah untuk melakukan tuning pada model yang overfit daripada yang underfit.

# Dataset Cross Validation Checking

In [10]:
from sklearn.model_selection import cross_val_score

In [11]:
def find_CVS(features,target,model, partition, scoring_system):
    X_train, X_test, y_train, y_test = train_test_split(features,target,random_state=11111992)
    classification = model()
    score = cross_val_score(classification,features, target, cv=partition, scoring=scoring_system).mean()
    return score

In [12]:
def cv_score_accumulation(algorithm_list, X, y, partition, scoring_system, notes):
    cv_score = []
    notes_arr = []
    for i in algorithm_list :
        score = find_CVS(X,y,i, partition, scoring_system)
        cv_score.append(score)
        notes_arr.append(notes)

    cv_df = pd.DataFrame({
        'Algorithm': algorithm_list,
        'Notes': notes_arr,
        'CV Score': cv_score
    })
    return cv_df

In [13]:
algorithm_list = [DecisionTreeClassifier,RandomForestClassifier, GradientBoostingClassifier]
a = cv_score_accumulation(algorithm_list, X, y, 5, 'recall', 'with Outliers')
b = cv_score_accumulation(algorithm_list, X_wo, y_wo, 5, 'recall', 'without Outliers')
# x = pd.concat([a, b],ignore_index=True)
# x.set_index('Algorithm').sort_values(by='CV Score', ascending=False)
a

,Algorithm,Notes,CV Score
0,<class 'sklearn.tree._classes.DecisionTreeClas...,with Outliers,0.380794
1,<class 'sklearn.ensemble._forest.RandomForestC...,with Outliers,0.175714
2,<class 'sklearn.ensemble._gb.GradientBoostingC...,with Outliers,0.307143


In [14]:
b

,Algorithm,Notes,CV Score
0,<class 'sklearn.tree._classes.DecisionTreeClas...,without Outliers,0.339921
1,<class 'sklearn.ensemble._forest.RandomForestC...,without Outliers,0.188933
2,<class 'sklearn.ensemble._gb.GradientBoostingC...,without Outliers,0.330830


**Temuan** : Dengan split sebanyak 5 dan recall-scoring, ditemukan bahwa urutan algoritma tidak berubah dengan ada atau tidaknya outlier.

Adapun saya memilih dataset dengan outliers karena pertimbangan berikut:
- Dengan memilih dataset tanpa outlier, maka jumlah rows berkurang sangat banyak. Dan ini berpengaruh dengan karakteristik data.
- Pada EDA, meskipun dianggap univariate outlier, namun secara keseluruhan ia bukan multivariate outliers (based on Mahalanobis Distance)
- According to the rank, dengan adanya outliers ataupun tidak, tidak membuat urutan algoritma tersebut berubah. 

Maka, selanjutnya kita akan menggunakan algoritma DecisionTree, RandomForest, juga GradientBoosting menggunakan dataset dengan outliers untuk Hyperparameter Tuning.

# Hyperparameter Tuning

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
import numpy as np

In [16]:
algorithm_list = [
    DecisionTreeClassifier,
    RandomForestClassifier,
    GradientBoostingClassifier
]
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=11111992)

def pipelining(algorithm):
    model_pipeline = Pipeline([
        ('pca', PCA()),
        ('algorithm', algorithm())
    ])
    return model_pipeline

In [17]:
params_for_DecisionTreeClassifier = {
    'pca__n_components': [10,15,20],
    'algorithm__random_state': [11111992],
    'algorithm__splitter' : ['best', 'random'],
    'algorithm__max_features' : list(range(1,X_train.shape[1])),
#     'algorithm__class_weight': [{0: 1, 1: 4.75}], ----> Bisa dicoba di laptop
#     'algorithm__class_weight': [{0: x, 1: 1.0-x} for x in np.linspace(0.05, 0.95, 20)], ----> Terlalu berat
#     'algorithm__class_weight': [{0: 0.75, 1: 0.25}, {0: 0.7, 1: 0.3}, {0: 0.8, 1: 0.2}], ---> Masih berat
    'algorithm__class_weight': [{0: 1, 1: 4.75}],
    'algorithm__max_depth' : np.linspace(4, 15, 10),
    'algorithm__min_samples_split' : [200,300,400],
    'algorithm__min_samples_leaf' : [100,150,200],
}

params_for_RandomForestClassifier = {
    'pca__n_components': [10,15,20],
    'algorithm__n_estimators': [100,300,500,700],
    'algorithm__random_state': [11111992],
#     'algorithm__max_features' : list(range(1,X_train.shape[1])),
    'algorithm__class_weight': [{0: 1, 1: 4.75}],
    'algorithm__max_depth' : np.linspace(4, 15, 10),
    'algorithm__min_samples_split' : [200,300,400],
    'algorithm__min_samples_leaf' : [100,150,200],
}

params_for_GradientBoostingClassifier = {
    'pca__n_components': [10,15,20],
#     'algorithm__criterion': ["friedman_mse",  "mae"],
#     'algorithm__loss':["deviance","exponential"],
    'algorithm__random_state': [11111992],
    'algorithm__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3, 0.5],
    'algorithm__n_estimators': [100,400,800],
#     'algorithm__min_samples_split' : [200,300,400],
#     'algorithm__min_samples_leaf' : [100,150,200],
    'algorithm__max_depth' : np.linspace(4, 20, 4),
    'algorithm__subsample': [0.7,0.8],
}

In [18]:
params_list = [
    params_for_DecisionTreeClassifier,
    params_for_RandomForestClassifier,
    params_for_GradientBoostingClassifier
]

grid_list = []

for i in range(3):
    grid = GridSearchCV(pipelining(algorithm_list[i]),
                        params_list[i],
                        cv=5, scoring='recall', n_jobs=-1, verbose=2)
    print(algorithm_list[i])
    grid.fit(X_train, y_train)
    grid_list.append(grid)

<class 'sklearn.tree._classes.DecisionTreeClassifier'>
Fitting 5 folds for each of 23220 candidates, totalling 116100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 656 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 2280 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done 4544 tasks      | elapsed:   33.2s
[Parallel(n_jobs=-1)]: Done 7464 tasks      | elapsed:   48.2s
[Parallel(n_jobs=-1)]: Done 11024 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 15240 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 20096 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 25608 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 31760 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 38568 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 46016 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 54120 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 62864 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 72264 t

<class 'sklearn.ensemble._forest.RandomForestClassifier'>
Fitting 5 folds for each of 1080 candidates, totalling 5400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   43.9s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed: 16.0min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 19.8min
[Parallel(n_jobs=-1)]: Done 4893 tasks      | elapsed: 24.0min
[Parallel(n_jobs=-1)]: Done 5400 out of 5400 | elapsed: 26.5min finished


<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>
Fitting 5 folds for each of 432 candidates, totalling 2160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 29.7min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 43.4min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed: 54.8min
[Parallel(n_jobs=-1)]: Done 2160 out of 2160 | elapsed: 60.5min finished


In [19]:
%%script false --no-raise-error
grid.fit(X_train, y_train) # for DecisionTreeClassifier()

In [20]:
print('################################################################################################### Report')

################################################################################################### Report


In [21]:
for i in range(3):
    print('-----------------------------')
    print(algorithm_list[i])
    print(grid_list[i].best_estimator_)

-----------------------------
<class 'sklearn.tree._classes.DecisionTreeClassifier'>
Pipeline(steps=[('pca', PCA(n_components=10)),
                ('algorithm',
                 DecisionTreeClassifier(class_weight={0: 1, 1: 4.75},
                                        max_depth=4.0, max_features=1,
                                        min_samples_leaf=100,
                                        min_samples_split=200,
                                        random_state=11111992,
                                        splitter='random'))])
-----------------------------
<class 'sklearn.ensemble._forest.RandomForestClassifier'>
Pipeline(steps=[('pca', PCA(n_components=20)),
                ('algorithm',
                 RandomForestClassifier(class_weight={0: 1, 1: 4.75},
                                        max_depth=5.222222222222222,
                                        min_samples_leaf=150,
                                        min_samples_split=200, n_estimators=300,


In [22]:
grid_list[0].best_params_

{'algorithm__class_weight': {0: 1, 1: 4.75},
 'algorithm__max_depth': 4.0,
 'algorithm__max_features': 1,
 'algorithm__min_samples_leaf': 100,
 'algorithm__min_samples_split': 200,
 'algorithm__random_state': 11111992,
 'algorithm__splitter': 'random',
 'pca__n_components': 10}

In [23]:
grid_list[1].best_params_

{'algorithm__class_weight': {0: 1, 1: 4.75},
 'algorithm__max_depth': 5.222222222222222,
 'algorithm__min_samples_leaf': 150,
 'algorithm__min_samples_split': 200,
 'algorithm__n_estimators': 300,
 'algorithm__random_state': 11111992,
 'pca__n_components': 20}

In [24]:
grid_list[2].best_params_

{'algorithm__learning_rate': 0.5,
 'algorithm__max_depth': 4.0,
 'algorithm__n_estimators': 100,
 'algorithm__random_state': 11111992,
 'algorithm__subsample': 0.7,
 'pca__n_components': 20}

In [25]:
for i in range(3):
    print('-----------------------------')
    print(algorithm_list[i])
    print(grid_list[i].best_score_)

-----------------------------
<class 'sklearn.tree._classes.DecisionTreeClassifier'>
0.8649572649572651
-----------------------------
<class 'sklearn.ensemble._forest.RandomForestClassifier'>
0.5914529914529914
-----------------------------
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>
0.3188034188034188


In [26]:
print(algorithm_list[0])
y_pred = grid_list[0].predict(X_train)
print(classification_report(y_train, y_pred))
y_pred = grid_list[0].predict(X_test)
print(classification_report(y_test, y_pred))

<class 'sklearn.tree._classes.DecisionTreeClassifier'>
              precision    recall  f1-score   support

           0       0.89      0.23      0.37       639
           1       0.19      0.86      0.31       132

    accuracy                           0.34       771
   macro avg       0.54      0.54      0.34       771
weighted avg       0.77      0.34      0.36       771

              precision    recall  f1-score   support

           0       0.97      0.28      0.43       214
           1       0.21      0.95      0.35        44

    accuracy                           0.39       258
   macro avg       0.59      0.62      0.39       258
weighted avg       0.84      0.39      0.42       258



In [27]:
print(algorithm_list[1])
y_pred = grid_list[1].predict(X_train)
print(classification_report(y_train, y_pred))
y_pred = grid_list[1].predict(X_test)
print(classification_report(y_test, y_pred))

<class 'sklearn.ensemble._forest.RandomForestClassifier'>
              precision    recall  f1-score   support

           0       0.91      0.78      0.84       639
           1       0.37      0.63      0.47       132

    accuracy                           0.75       771
   macro avg       0.64      0.70      0.65       771
weighted avg       0.82      0.75      0.78       771

              precision    recall  f1-score   support

           0       0.91      0.83      0.87       214
           1       0.42      0.61      0.50        44

    accuracy                           0.79       258
   macro avg       0.67      0.72      0.68       258
weighted avg       0.83      0.79      0.80       258



In [28]:
print(algorithm_list[2])
y_pred = grid_list[2].predict(X_train)
print(classification_report(y_train, y_pred))
y_pred = grid_list[2].predict(X_test)
print(classification_report(y_test, y_pred))

<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       639
           1       1.00      1.00      1.00       132

    accuracy                           1.00       771
   macro avg       1.00      1.00      1.00       771
weighted avg       1.00      1.00      1.00       771

              precision    recall  f1-score   support

           0       0.85      0.91      0.88       214
           1       0.34      0.23      0.27        44

    accuracy                           0.79       258
   macro avg       0.60      0.57      0.58       258
weighted avg       0.77      0.79      0.78       258



In [29]:
print('###################################################################################################### End of Report')

###################################################################################################### End of Report


**Temuan** :
- DecisionTreeClassifier memerlukan waktu yang lebih singkat (12 menit), namun mendapatkan score recall yang lebih baik daripada RandomForestClassifier (27 menit) dan juga GradientBoosting (61 menit)

Oleh karenanya, selanjutnya saya akan menggunakan DecisionTreeClassifier dan RandomForestClassifier pada dataset dengan outlier. Akan tetapi, di notebook selanjutnya akan saya lakukan GridSearchCV sekali lagi pada 2 model ini dengan parameter yang mungkin sedikit lebih banyak daripada pada notebook ini.